In [2]:

import random
import os
import hashlib
import base64
import numpy as np
import math

## $\text{Parte I}$

### Funções utilitárias

In [3]:
Key = tuple[int, int]

def is_prime(n, _precision_for_huge_n=16):
    """Teste de primalidade de Miller-Rabin. Detalhes: https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python:_Probably_correct_answers"""
    def is_prime_trial_division(n: int) -> bool:
        '''Test if a given integer n is a prime number using trial division'''
        if n == 2:
            return True
        if n < 2 or n % 2 == 0:
            return False
        for i in range(3, math.ceil(math.sqrt(n)), 2):
            if n % i == 0:
                return False
        return True
    
    _known_primes = [2] + \
        [x for x in range(3, 1000, 2) if is_prime_trial_division(x)]

    def _try_composite(a, d, n, s):
        if pow(a, d, n) == 1:
            return False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                return False
        return True # n  is definitely composite
 
    if n in _known_primes:
        return True
    if any((n % p) == 0 for p in _known_primes) or n in (0, 1):
        return False
    d, s = n - 1, 0
    while not d % 2:
        d, s = d >> 1, s + 1
    # Returns exact according to http://primes.utm.edu/prove/prove2_3.html
    if n < 1373653: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3))
    if n < 25326001: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5))
    if n < 118670087467: 
        if n == 3215031751: 
            return False
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7))
    if n < 2152302898747: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11))
    if n < 3474749660383: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13))
    if n < 341550071728321: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17))
    # otherwise
    return not any(_try_composite(a, d, n, s) 
                   for a in _known_primes[:_precision_for_huge_n])


def generate_big_prime_pair(b):
    p = random.getrandbits(b)
    q = random.getrandbits(b)

    while not is_prime(p):
        p = random.getrandbits(b)
    
    while not is_prime(q) or p == q:
        q = random.getrandbits(b)

    return (p, q)   

def I2OSP(x: int, l: int):
    """Integer-to-Octet-String, responsável por converter um inteiro em BASE64.
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    
    # conversão para base 256 com tamanho l
    try:
        return x.to_bytes(l, byteorder='big')
    except OverflowError:
        return x.to_bytes((l+7)//8, byteorder='big')
        

def OS2IP(X: bytes):
    """Octet-String-to-Integer, responsável por converter o texto BASE64 em um inteiro
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    return int.from_bytes(X, byteorder='big')

def bitwise_xor(a: bytes, b: bytes):
    """Operação XOR entre dois bytes"""
    r=b""
    
    for i in range(max(len(a), len(b))):
        if i >= len(a):
            r += b[i].to_bytes(1, byteorder='big')
        elif i >= len(b):
            r += a[i].to_bytes(1, byteorder='big')
        else:
            r += (a[i] ^ b[i]).to_bytes(1, byteorder='big')
    
    return r

def key_len(key: Key):
    """Retorna o número de octetos do modulo n da chave"""
    _, n = key
    return n.bit_length() // 8

### Geração de chaves

In [4]:

def generate_public_key(phi: int):
    # A chave pública e é um número primo tal que 1 < e < φ(n) e mdc(e, φ(n)) = 1
    e = random.randrange(2, phi)
    
    while not is_prime(e):
        e = random.randrange(2, phi)

    return e

def generate_private_key(e: int, phi: int):
    # A chave privada d é um número tal que e*d ≡ 1 mod φ(n)
    d = pow(e, -1, phi)
    
    return d

def generate_keypair() -> tuple[Key, Key]:
    while True:
        try :
            p, q = generate_big_prime_pair(1024)
            
            n = p * q
            phi = (p - 1) * (q - 1)
            
            e = generate_public_key(phi)
            d = generate_private_key(e, phi)
            break
        except:
            continue
    
    return ((e, n), (d, n))

### RSA

In [5]:
def RSA_encode(key: Key, m: bytes):
    e, n = key
    
    M = OS2IP(m)
    
    return pow(M, e, n)

def RSA_decode(key: Key, c: int):
    d, n = key
    
    return pow(c, d, n)

### Padding e criptografia e descriptografia
Detalhes: https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.3.

In [6]:

def MGF(Z: bytes, l: int):
    """MGF utilizando o Hash SHA-3"""
    
    # hLen denota o comprimento nos octetos da saída da função hash
    hLen = hashlib.sha3_256().digest_size
    
    if l > 2**32 * hLen:
        raise ValueError("Máscara muito longa.")
    
    T = b""
    
    for i in range(math.ceil(l / hLen)):
        # Converte i em um octeto C de tamanho 4 com a primitiva I2OSP.
        C = I2OSP(i, 4)
        # Concatena o resultado do hash SHA-3 de seed Z e C com T.
        T += hashlib.sha3_256(Z + C).digest()
    
    # A máscara M é a string consistindo dos primeiros l octetos de T.
    return T[:l]

def OAEP_encode(M: bytes, emLen: int, P: bytes = b""):
    """Encode de OAEP utilizando o Hash SHA-3"""

    hLen = hashlib.sha3_256().digest_size
    mLen = len(M)
    
    if mLen > emLen - 2 * hLen - 2:
        raise ValueError("Mensagem muito grande.")
    
    # Geração de uma string de octetos de comprimento (emLen - mLen - 2 * hLen - 1) consistindo de zeros.
    # Erro de overflow: "cannot fit 'int' into an index-sized integer"
    PS = b"\x00" * (emLen - mLen - 2 * hLen - 2)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Concatenação de pHash, PS, um octeto 0x01 e a mensagem M no bloco de dados DB.
    DB = pHash + PS + b"\x01" + M
    
    seed = random.getrandbits(hLen * 8).to_bytes(hLen, byteorder='big')
    
    # Mascaramento do bloco de dados
    dbMask = MGF(seed, emLen - hLen - 1)
    maskedDB = bitwise_xor(DB, dbMask)

    # Mascaramento da seed
    seedMask = MGF(maskedDB, hLen)
    maskedSeed = bitwise_xor(seed, seedMask)
    
    return b'\x00' + maskedSeed + maskedDB

def OAEP_decode(EM: bytes, P: bytes = b""):
    """Decode de OAEP utilizando o Hash SHA-3"""
    hLen = hashlib.sha3_256().digest_size
    emLen = len(EM)
    
    if emLen < 2 * hLen + 2:
        raise ValueError("Erro na decodificação. Tamanho de EM inválido.")
    
    maskedSeed = EM[1: 1 + hLen]
    maskedDB = EM[1 + hLen:]
    
    seedMask = MGF(maskedDB, hLen)
    seed = bitwise_xor(maskedSeed, seedMask)
    
    dbMask = MGF(seed, emLen - hLen - 1)
    DB = bitwise_xor(maskedDB, dbMask)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Separa a mensagem supondo o formato pHash_ || PS || 01 || M
    pHash_ = DB[:hLen]        
    
    if pHash != pHash_:
        raise ValueError("Erro na decodifiocação.")
    
    # Busca o byte 01 após o padding.
    i = hLen
    while DB[i] == 0:
        i += 1
    
    if DB[i] != 1:
        raise ValueError("Byte 0x01 não encontrado.")
    
    return DB[i+1:]


### Criptografia e descriptografia

In [7]:
def encrypt(key: Key, M: bytes, P: bytes = b""):
    """"Criptografa a mensagem usando RSA-OAEP"""
    try:
        e, _ = key
        
        emLen = key_len(key)
        
        EM = OAEP_encode(M, emLen, P)

        c = RSA_encode(key, EM)
        
        C = I2OSP(c, emLen)
        
        return C
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None
    
def decrypt(key: Key,  C: bytes, P: bytes = b""):
    """"Descriptografa a mensagem usando RSA-OAEP"""
    try:
        emLen = key_len(key)
        
        c = OS2IP(C)
        
        m = RSA_decode(key, c)
        
        EM = I2OSP(m, emLen)

        M = OAEP_decode(EM, P)
        
        return M
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None

    

## Teste

In [8]:
public_key, private_key = generate_keypair()

print(f"Chave pública: {base64.b64encode(str(public_key[0]).encode()).decode()}")
print(f"Chave privada: {base64.b64encode(str(private_key[0]).encode()).decode()}")

M = base64.b64encode(b"Eu te amo!")

print(f"Mensagem original: {M}")

while(True):
    try:
        C = encrypt(public_key, M)

        M_ = decrypt(private_key, C)
        
        print(f"Mensagem cifrada (base64): {base64.b64encode(C)}")

        break
    except:
        continue

print(f"Mensagem decifrada: {M_}")

if M == M_:
    print("Sucesso!")
else:
    print("Falha!")

Chave pública: MTA0MTk0NDM3OTk2NjcyMTQwMDE4ODQ4Mjk1MDAzOTk2Mjg0MzI2NTQ3NjgyMjA4OTM0NDc1NTQ4MTA5OTU0NTQ3NTcyNTc2MjkwMjc1NTY3ODI2MDk4MTE3NTAwMDgwMTk2NDg4MTA0MTc5OTUyMDYyMzE3MDUyNDg3MDg5MTQwMTQ4NDA5NDg4MDY4NDUzNTcwNzQxMDk1MjY4OTg0NTg3NzkwMzI3MTU2NzQ5MDQxNTAxMzQ1MzMyODc1MTU0NTYwODA3NTI4MjQyMTM2NTc0MDM2NTQzMDY3NzM4NDkzNzE4NTk1ODg4NzM1ODEwMjUwNzgwNDk0NTgyOTQ1NTg2NzE1ODUzNDE2Mjg1NjE5OTI4OTk0MzkxMzQ0ODY2NDk5NTk5NDEwNTgzMDgyODQzNjQxMDQ0ODExOTgwNzE0NDUwNzIyMzc1NDAxMjA2MTUwOTAyNTEwMTY0ODQ5Mzk3MTY0MDA4NDk3NjQ4MzUxOTI4OTYwMjQ1Mzc3NzQ3MDkwNTIxNzIwMjAyMDkwNzA5NTkxMjk2MTI2Mjc4ODY3MDI5NjQyMDc4NDgxMzAwNDI5NTE3MTI1NTU2MDY2OTI3MTM0MTI3NTI1OTU3NDIzMDI0NDQ2ODkxNzM5NDQzMzgzMjc2NzQ2OTAzMTI1Mjc1NjI5NjE1MzMyNjAzMTY0ODE5ODgxNjM1OTA0MjY3NTY4NTQyNDc3NDUzMTg0NjIyNzU0MTkyODQwMjU3OTQyODEyNjg5NTY4ODA0NjIyMTU4MDUzMTU3NTY1Njc1ODE4MjE0NDc4MDQzNzcyMjE5MjU0MTYyMTM=
Chave privada: NTc5NDc0MjU1MjU0OTU4MDU1NTYyODgwNzkyOTI1MjQ3NTQ3NjU3NDgyODkzOTk5ODcxMjI2NDAzMDkyMTg1ODIzMjg3ODQ3MTg2ODQ5ODYxOTE5NTY5Mjc4NzI0MDg2OTIxOTY5OTQ3MDUwN

## Criptografia + Assinatura de arquivo

In [20]:
with open("lorem.txt", "rb") as f:   
    M = base64.b64encode(f.read())
    while(True):
        try:
            C = encrypt(public_key, M)
           
            break
        except:
            continue
    
    with open("lorem.enc.txt", "wb") as f:
        f.write(C)

In [21]:
with open("lorem.enc.txt", "rb") as f:  
    C = f.read()
    while(True):
        try:
            M = base64.b64decode(decrypt(private_key, C))
           
            break
        except:
            continue

    with open("lorem.dec.txt", "wb") as f:
        f.write(M)